<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/Experimenting_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Packages and Dataset

In [1]:
!pip install timm -q
!wget https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Parasite.7z
!7z x /content/Parasite.7z

     |████████████████████████████████| 296kB 8.3MB/s 
--2021-03-19 13:26:51--  https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Parasite.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20003284 (19M) [application/octet-stream]
Saving to: ‘Parasite.7z’

Parasite.7z         100%[===================>]  19.08M  94.9MB/s    in 0.2s    

2021-03-19 13:26:51 (94.9 MB/s) - ‘Parasite.7z’ saved [20003284/20003284]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 20003284 bytes (20 MiB)

Extr

# Importing Packages

In [2]:
import torch
import timm
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
import time
import pandas as pd
import torchvision.transforms.functional as F
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import random_split
import copy
from sklearn.metrics import confusion_matrix,classification_report

device = torch.device('cuda')

In [9]:
data_transforms = {
    'advanced': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        # transforms.
        # transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomRotation(90),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'basic': transforms.Compose([
        transforms.Resize((224,224)),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Dataset Preprocessing

In [3]:
df=pd.read_csv("/content/Dataset/all_meta_data_updated.csv")
df.species=df.species.replace({"Hymenolepis diminuta":"Hymenolepis spp",
                    "Hymenolepis nana":"Hymenolepis spp",
                    "Taenia solium":"na",
                    "Clonorchis sinensis":"na",
                    "Trichinella sp.":"na",
                    "B. coli":"na",
                    "Toxoplasma gondii":"na",
                    })

df['phylum'] = df['phylum'].replace({'Roundworms':'Nematoda','Flatworm':'Platyhelminthes'})
df['class'] = df['class'].replace({'\tConoidasida':'Conoidasida','Tapeworms':'Cestoda'})
df['species'] = df['species'].replace({'Ancyclostoma sp.':'Ancyclostoma sp.','Entamoeba histolytica]':'Entamoeba histolytica'})
df['form'] = df['form'].replace({'rings':'trophozoites','decorticated egg':'egg'})
df['sample'] = df['sample'].replace({'intestinal tissue':'intestine', 'intestine':'intestine'})

df = df[df.species!="na"].reset_index(drop=True)
def get_train_val(df,val_size=0.25):
    l=np.array(df.index)
    np.random.shuffle(l)
    val_i=int(l.shape[0]*val_size)
    return l[val_i:],l[:val_i]

## Selecting first 19 labels

In [4]:
df = [x for _,x in df.groupby('species') if len(x)>11] # Same species labels as in ensta_colab2.ipynb
df = pd.concat(df,ignore_index=True)

df.to_csv("/content/all_meta_data_updated.csv",index=False)

## Dataset Class

In [5]:
#Defining custom dataset
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self,csv_file,idx,transform=None):
        self.meta_data = pd.read_csv(csv_file).iloc[idx,:].reset_index(drop=True)
        # print(self.meta_data.head())
        self.transform = transform
        self.labels=["phylum","class","species","form","sample"]
        self.label_encoder=self.get_label_encode()
        self.nclasses=self.get_nclasses()

    def __len__(self):
        return len(self.meta_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.meta_data.loc[idx,"img_path"]
        image = plt.imread(img_path)
        image = F.to_pil_image(image)
        labels = dict(self.meta_data.loc[idx,self.labels])

        if self.transform:
            image = self.transform(image)
                
        dict_data = {
        'image': image,
        'labels': labels
        }
        return dict_data
    
    def get_label_encode(self):
        d=dict()
        for label in self.labels:            
            le = LabelEncoder()
            self.meta_data[label]=le.fit_transform(self.meta_data[label])
            d[label]=le
        return d
    
    def get_nclasses(self):
        d=dict()
        for label in self.label_encoder:
            d[label]=len(self.label_encoder[label].classes_)
        return d

# NN model

In [6]:
class DNN1(nn.Module):
    def __init__(self,resnet_path,nclasses,labels):
        super(DNN1, self).__init__()
        #number of multi-labels
        self.labels=labels
        #no of out_features forr each label
        self.nclasses=nclasses
        self.nmid_ft=512
        
        #Resnet model
        self.resnet= timm.create_model('resnet50', pretrained=True)
        #input channels of our classifier
        in_features = self.resnet.fc.in_features
        self.dropout=nn.Dropout(0.2)
        
        #Removing the pretrained classifier layer and freezing the weights
        self.resnet = nn.Sequential(*(list(self.resnet.children())[:-1]))
        for param in self.resnet.parameters():
            param.requires_grad = False
            
        #building our own classifier for each label
        self.fc1=nn.Linear(in_features,self.nmid_ft)
        
        
        self.classifier = nn.ModuleDict()
        for label in labels:
            self.classifier[label]=nn.Linear(self.nmid_ft,nclasses[label])
            
    def forward(self,x):
        """
        for the forward pass of the model, the input image will pass through resnet FCNN
        then it will be flattened and there will be n parallel Dense layer for each label.
        then loss function will be crossentropy which has inbuilt softmax function
        """
        x=torch.relu(self.fc1(self.resnet(x).view(x.shape[0],-1)))
        x=self.dropout(x)
        y=dict()
        for label in self.classifier.keys():
            y[label]=self.classifier[label](x)
        return y

# Data splitting
Choose any of the given cell

In [10]:
from sklearn.model_selection import train_test_split
t,v = train_test_split(df,test_size = 0.25,stratify = df['species'])
train_dataset=Dataset("/content/all_meta_data_updated.csv",idx=t.index, transform=data_transforms["advanced"])

val_dataset=Dataset("/content/all_meta_data_updated.csv",idx=v.index, transform=data_transforms["basic"])

print(train_dataset.nclasses)
print(train_dataset.labels)
print(val_dataset.nclasses)
print(val_dataset.labels)

{'phylum': 5, 'class': 7, 'species': 19, 'form': 11, 'sample': 10}
['phylum', 'class', 'species', 'form', 'sample']
{'phylum': 5, 'class': 7, 'species': 19, 'form': 10, 'sample': 9}
['phylum', 'class', 'species', 'form', 'sample']


In [ ]:
trainidx,validx=get_train_val(df,val_size=0.25)
train_dataset=Dataset("/content/all_meta_data_updated.csv",idx=trainidx, transform=data_transforms["advanced"])

val_dataset=Dataset("/content/all_meta_data_updated.csv",idx=validx, transform=data_transforms["basic"])

print(train_dataset.nclasses)
print(train_dataset.labels)
print(val_dataset.nclasses)
print(val_dataset.labels)

In [11]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=32,shuffle=True,)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset),shuffle=True,)

# Metrics and training functions

In [12]:
#appending all the loss for each label
def get_loss(out,labels,loss_fn):
    loss=0
    for label in labels:
        loss+=loss_fn(out[label],labels[label])
    return loss/len(labels)


def cal_accuracy(out,labels,batch_size,epoch_acc):
    for key in out:
        q=(out[key].detach().argmax(axis=1)==labels[key]).sum().item()/batch_size
        if not key in epoch_acc.keys():
            epoch_acc[key]=q
        else:
            epoch_acc[key]+=q
    return epoch_acc


def get_avg_acc(epoch_acc,loader):
    acc=0
    for key in epoch_acc:
        epoch_acc[key]/=len(loader)
        acc+=epoch_acc[key]
        
    return acc/len(epoch_acc),epoch_acc

def evaluate(model,loss_fn,loader):
    # print(device)
    model.eval()
    epoch_acc={}
    epoch_loss=0
    for batch in loader:
        img=batch["image"].to(device)
        labels=batch["labels"]
        for key in labels:
            labels[key]=labels[key].to(device)
        
        out=model(img)
        loss=get_loss(out,labels,loss_fn)
        epoch_loss+=loss.detach().item()
        epoch_acc=cal_accuracy(out,labels,img.shape[0],epoch_acc)
        
    avg_acc,epoch_acc=get_avg_acc(epoch_acc,loader)
    return (epoch_loss/len(loader),avg_acc,epoch_acc)


def train_model(model, loss_fn, opt,trainloader,valloader, num_epochs=1):
    since = time.time()
    best_model=""
    max_acc=-999
    train={"loss":[],"avg":[],"individual":[]}
    val={"loss":[],"avg":[],"individual":[]}
    for epoch in range(num_epochs):
        print('Epoch {} of {}'.format(epoch+1, num_epochs))
        epoch_loss=0
        epoch_acc={}
        model.train()
        # Iterate over data.
        for i,data in enumerate(trainloader):
            opt.zero_grad()
            img=data["image"].to(device)
            labels=data["labels"]
            for key in labels:
                labels[key]=labels[key].to(device)
            

            out=model(img)
            loss=get_loss(out,labels,loss_fn)
            epoch_loss+=loss.detach().item()
            loss.backward()
            opt.step()
            epoch_acc=cal_accuracy(out,labels,img.shape[0],epoch_acc)
            
        val_result=evaluate(model,loss_fn,valloader)
        avg_acc,epoch_acc=get_avg_acc(epoch_acc,trainloader)
        avg_loss=epoch_loss/len(trainloader)
        print('Train')
        print(f"Loss: {avg_loss}\nAccuracy: Each_label_acc->{epoch_acc}\n\tAvg acc->{avg_acc}\n")
        print("Validation")
        print(f"Loss: {val_result[0]}\nAccuracy: Each_label_acc->{val_result[2]}\n\tAvg acc->{val_result[1]}")
        print("-"*50)
        
        train["avg"].append(avg_acc)
        train["individual"].append(epoch_acc)
        train["loss"].append(avg_loss)
        
        val["avg"].append(val_result[1])
        val["individual"].append(val_result[2])
        val["loss"].append(val_result[0])
        if val_result[1]>max_acc:
            max_acc=val_result[1]
            best_model=copy.deepcopy(model)

        
       

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return train,val,best_model,max_acc
# train,val=train_model(model, loss_fn, optimizer,trainloader,valloader,num_epochs=2)

# lets train

In [16]:
model=DNN1("drive/My Drive/resnet", train_dataset.nclasses, train_dataset.labels,).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train,val,best_model,best_acc=train_model(model, loss_fn, optimizer,trainloader,valloader,num_epochs=100)

Epoch 1 of 100
Train
Loss: 1.9751525250348179
Accuracy: Each_label_acc->{'phylum': 0.5004058441558441, 'class': 0.3965097402597403, 'species': 0.08400974025974026, 'form': 0.294237012987013, 'sample': 0.4675324675324676}
	Avg acc->0.34853896103896104

Validation
Loss: 1.9730643033981323
Accuracy: Each_label_acc->{'phylum': 0.6146788990825688, 'class': 0.47706422018348627, 'species': 0.1743119266055046, 'form': 0.23853211009174313, 'sample': 0.5045871559633027}
	Avg acc->0.40183486238532107
--------------------------------------------------
Epoch 2 of 100
Train
Loss: 1.5341215892271562
Accuracy: Each_label_acc->{'phylum': 0.6988636363636364, 'class': 0.583198051948052, 'species': 0.20900974025974026, 'form': 0.4411525974025974, 'sample': 0.5945616883116883}
	Avg acc->0.5053571428571428

Validation
Loss: 1.8941186666488647
Accuracy: Each_label_acc->{'phylum': 0.5688073394495413, 'class': 0.5504587155963303, 'species': 0.27522935779816515, 'form': 0.24770642201834864, 'sample': 0.48623853

In [ ]:
plt.plot(list(range(len(val["avg"][:-5]))),val["avg"][:-5])
plt.plot(list(range(len(train["avg"][:-5]))),train["avg"][:-5])
plt.title("Accuracy plot")
plt.xlabel("Epoch")
plt.ylabel("Avg Accuracy")
plt.legend(["val","train"])
m=max(val["avg"])
t=max(train["avg"])
plt.annotate(f"Max val acc: {m:0.2f}\nMax train acc: {t:0.2f}",(40,0.4))
# plt.annotate(f"Max train acc: {t:0.2f}",(30,0.0))
plt.savefig("Accuracy plot.jpg")

In [ ]:
df.species.value_counts()

Plasmodium falciparum        36
Plasmodium vivax             36
Cyclospora cayetanensis      30
Plasmodium malariae          30
Ascaris lumbricoides         29
Plasmodium ovale             27
Giardia duodenalis           27
Entamoeba histolytica        27
Strongyloides stercoralis    22
Dibothriocephalus latus      21
Hymenolepis spp              21
Trichuris Trichuria          19
Enterobius vermicularis      18
Ancyclostoma sp.             17
Toxocara canis               17
Cryptosporidium sp.          16
Sarcocystis sp.              16
Plasmodium knowlesi          15
Echinococcus granulosus      12
Name: species, dtype: int64

# Errors

In [ ]:
from collections import Counter

def get_attrs(out,labels):
    o=torch.ones((len(labels),len(labels["phylum"])))
    l=torch.ones((len(labels),len(labels["phylum"])))
    for i,key in enumerate(labels):
        o[i]=out[key].cpu().argmax(axis=1)
        l[i]=labels[key]
        
    return dict(Counter((o==l).sum(axis=0).numpy()))
# Counter(((l==o).sum(axis=0)))


def cal_accuracy(out,labels,batch_size,epoch_acc):
    for key in out:
        q=(out[key].detach().argmax(axis=1)==labels[key]).sum().item()/batch_size
        if not key in epoch_acc.keys():
            epoch_acc[key]=q
        else:
            epoch_acc[key]+=q
    return epoch_acc
    


def get_avg_acc(epoch_acc,loader):
    acc=0
    for key in epoch_acc:
        epoch_acc[key]/=len(loader)
        acc+=epoch_acc[key]
        
    return acc/len(epoch_acc),epoch_acc

def get_report(out,labels):
    c=0
    rep=""
    for key in labels:
        if c==0:
            c+=1
            r=classification_report(labels[key].cpu(),out[key].cpu().argmax(axis=1),target_names=val_dataset.label_encoder[key].classes_,output_dict=True)
            rep=pd.DataFrame(r).transpose()
            rep=rep.append(pd.Series(dtype=str,name=key)).append(pd.Series(dtype=str,name="")).append(pd.Series(dtype=str,name=""))
        else:
            r=classification_report(labels[key].cpu(),out[key].cpu().argmax(axis=1),target_names=val_dataset.label_encoder[key].classes_,output_dict=True)
            r=pd.DataFrame(r).transpose()
            rep=rep.append(r).append(pd.Series(dtype=str,name=key)).append(pd.Series(dtype=str,name="")).append(pd.Series(dtype=str,name=""))
    return rep
# report(out,labe)

def evaluate_test(model,loss_fn,loader):
    # print(device)
    model.eval()
    epoch_acc={}
    # epoch_loss=0

    for batch in loader:
        img=batch["image"].to(device)
        labels=batch["labels"]
        for key in labels:
            labels[key]=labels[key].to(device)
        
        out=model(img)
        epoch_acc=cal_accuracy(out,labels,img.shape[0],epoch_acc)
        report=get_report(out,labels)
        attrbs=get_attrs(out,labels)
        
    avg_acc,epoch_acc=get_avg_acc(epoch_acc,loader)
    return (avg_acc,epoch_acc,attrbs),report

r,report=evaluate_test(best_model,loss_fn=loss_fn,loader=valloader)

In [ ]:
r

In [ ]:
report